# Experimentation and Results

## Objective of the project 

This study seeks to conduct a thorough comparative analysis of these three models, focusing
on their performance with regards to accuracy, computational complexity, scalability, and their
effectiveness in handling data sparsity and dynamically changing environments. By evaluat-
ing these aspects, the research aims to illuminate the operational strengths and weaknesses
of each model, providing clear insights that could guide the development and deployment of
future recommender systems. Through this comparative framework, we aspire to answer which
model, under what conditions, provides the most reliable and robust recommendations, thereby
significantly contributing to the optimization of digital services.

In [13]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter, defaultdict
from surprise import Dataset, Reader, KNNBasic, SVD, CoClustering, accuracy
#from surprise.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from surprise.accuracy import rmse, mae

In [5]:
links_df = pd.read_csv('MovieLens_100k/links.csv')
movies_df = pd.read_csv('MovieLens_100k/movies.csv')
ratings_df = pd.read_csv('MovieLens_100k/ratings.csv')
tags_df = pd.read_csv('MovieLens_100k/tags.csv')

datasets = {
    "Links": links_df,
    "Movies": movies_df,
    "Ratings": ratings_df,
    "Tags": tags_df
}

datasets_info = {name: df.head() for name, df in datasets.items()}
datasets_info

{'Links':    movieId  imdbId   tmdbId
 0        1  114709    862.0
 1        2  113497   8844.0
 2        3  113228  15602.0
 3        4  114885  31357.0
 4        5  113041  11862.0,
 'Movies':    movieId                               title  \
 0        1                    Toy Story (1995)   
 1        2                      Jumanji (1995)   
 2        3             Grumpier Old Men (1995)   
 3        4            Waiting to Exhale (1995)   
 4        5  Father of the Bride Part II (1995)   
 
                                         genres  
 0  Adventure|Animation|Children|Comedy|Fantasy  
 1                   Adventure|Children|Fantasy  
 2                               Comedy|Romance  
 3                         Comedy|Drama|Romance  
 4                                       Comedy  ,
 'Ratings':    userId  movieId  rating  timestamp
 0       1        1     4.0  964982703
 1       1        3     4.0  964981247
 2       1        6     4.0  964982224
 3       1       47     5.0  9

## Dataset structure

In [6]:
# Check for missing values in each dataset
missing_values = {name: df.isnull().sum() for name, df in datasets.items()}

# Print the information about missing values
for name, missing in missing_values.items():
    print(f"Missing values in {name} dataset:\n{missing}\n")

Missing values in Links dataset:
movieId    0
imdbId     0
tmdbId     8
dtype: int64

Missing values in Movies dataset:
movieId    0
title      0
genres     0
dtype: int64

Missing values in Ratings dataset:
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

Missing values in Tags dataset:
userId       0
movieId      0
tag          0
timestamp    0
dtype: int64



In [7]:
# Print the shape of each DataFrame
for name, df in datasets.items():
    print(f"The shape of the {name} DataFrame is: {df.shape}")

The shape of the Links DataFrame is: (9742, 3)
The shape of the Movies DataFrame is: (9742, 3)
The shape of the Ratings DataFrame is: (100836, 4)
The shape of the Tags DataFrame is: (3683, 4)


In [8]:
distribution_of_ratings = ratings_df.groupby('rating').size().reset_index(name='count')
distribution_of_ratings

,rating,count
0,0.5,1370
1,1.0,2811
2,1.5,1791
3,2.0,7551
4,2.5,5550
5,3.0,20047
6,3.5,13136
7,4.0,26818
8,4.5,8551
9,5.0,13211


# Graph-based Algorithms

### For the graph-based models, we have implemented 3 algorithms as well: LightGCN, Graph Attention Network, GraphSAGE
Graph Construction: In the graph construction phase, nodes are created to represent both users
and movies, while edges represent the interactions between these users and movies based on
their ratings. To facilitate this, numpy is utilized for constructing the adjacency matrix, which
captures the user-movie interaction graph in a structured form. This adjacency matrix serves
as the foundation for various graph-based algorithms, allowing us to represent the relationships
between users and movies effectively. Additionally, TensorFlow is employed to handle the graph
representation and computation, leveraging its powerful capabilities for efficient processing and
model training in subsequent stages. 

### In this first part of the code, we have implemented all the useful functions and code that all the 3 algorithms shared in commun to avoid code redundancy. 

In [45]:
import numpy as np
import pandas as pd
import tensorflow as tf
import time
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder

# Load the dataset
ratings_df = pd.read_csv('MovieLens_100k/ratings.csv')
movies_df = pd.read_csv('MovieLens_100k/movies.csv')

# Data Preprocessing
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

ratings_df['user'] = user_encoder.fit_transform(ratings_df['userId'])
ratings_df['item'] = item_encoder.fit_transform(ratings_df['movieId'])

num_users = ratings_df['user'].nunique()
num_items = ratings_df['item'].nunique()


def evaluate_model(model, train_data, test_data, adj_matrix=None):
    # Prepare the training data
    user_indices = train_data['userId'].values
    item_indices = train_data['movieId'].values
    labels = train_data['rating'].values

    dataset = tf.data.Dataset.from_tensor_slices((user_indices, item_indices, labels))
    dataset = dataset.shuffle(buffer_size=len(train_data)).batch(256)

    # Training loop
    optimizer = tf.keras.optimizers.Adam()
    loss_fn = tf.keras.losses.MeanSquaredError()  

    @tf.function
    def train_step(user_indices, item_indices, labels):
        with tf.GradientTape() as tape:
            if adj_matrix is not None:
                scores = model(user_indices, item_indices, adj_matrix)
            else:
                scores = model(user_indices, item_indices)
            loss = loss_fn(labels, scores)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return loss

    num_epochs = 10
    start_time = time.time()

    for epoch in range(num_epochs):
        for batch in dataset:
            user_indices_batch, item_indices_batch, labels_batch = batch
            loss = train_step(user_indices_batch, item_indices_batch, labels_batch)
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')

    training_time = time.time() - start_time

    # Evaluating the model
    predictions = []

    test_user_indices = test_data['user'].values
    test_item_indices = test_data['item'].values
    test_labels = test_data['rating'].values

    for (user_index, item_index, label) in zip(test_user_indices, test_item_indices, test_labels):
        user_index_tensor = tf.constant([user_index])
        item_index_tensor = tf.constant([item_index])
        if adj_matrix is not None:
            score = model(user_index_tensor, item_index_tensor, adj_matrix).numpy()[0]
        else:
            score = model(user_index_tensor, item_index_tensor).numpy()[0]
        predictions.append((user_index, item_index, label, score, 0))

    return predictions, training_time

def compute_metrics(predictions):
    def get_top_n(predictions, n=10):
        top_n = defaultdict(list)
        for uid, iid, true_r, est, _ in predictions:
            top_n[uid].append((iid, est))
        for uid, user_ratings in top_n.items():
            user_ratings.sort(key=lambda x: x[1], reverse=True)
            top_n[uid] = user_ratings[:n]
        return top_n

    def precision_recall_at_k(predictions, k=10, threshold=0.7): 
        user_est_true = defaultdict(list)
        for uid, _, true_r, est, _ in predictions:
            user_est_true[uid].append((est, true_r))
        precisions = dict()
        recalls = dict()
        for uid, user_ratings in user_est_true.items():
            user_ratings.sort(key=lambda x: x[0], reverse=True)
            n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
            n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
            n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])
            precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
            recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
        return precisions, recalls

    def compute_mse(predictions):
        """Compute Mean Squared Error (MSE)."""
        mse = np.mean([(true_r - est) ** 2 for (_, _, true_r, est, _) in predictions])
        return mse

    
    def compute_rmse(predictions):
        """Compute Root Mean Squared Error (RMSE)."""
        mse = compute_mse(predictions)
        rmse = np.sqrt(mse)
        return rmse

    def compute_mae(predictions):
        mae = np.mean([abs(true_r - est) for (_, _, true_r, est, _) in predictions])
        return mae

    mse = compute_mse(predictions)
    rmse = compute_rmse(predictions)
    mae = compute_mae(predictions)
    precisions, recalls = precision_recall_at_k(predictions, k=10)

    precision_at_10 = np.mean(list(precisions.values()))
    recall_at_10 = np.mean(list(recalls.values()))

    return mse,rmse, mae, precision_at_10, recall_at_10

# Function to run evaluation for each model and scenario
def run_evaluation(model, model_name, train_data, test_data, adj_matrix=None):
    predictions, training_time = evaluate_model(model, train_data, test_data, adj_matrix)
    mse,rmse, mae, precision_at_10, recall_at_10 = compute_metrics(predictions)
    results = pd.DataFrame({
        "Scenario": [scenario],
        "Algorithm": [model_name],
        "MSE": [mse],
        "RMSE": [rmse],
        "MAE": [mae],
        "Precision@10": [precision_at_10],
        "Recall@10": [recall_at_10],
        "Running Time (s)": [training_time]
    })
    return results

## LightGCN Algorithms

In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
import time
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load the dataset
ratings_df = pd.read_csv('MovieLens_100k/ratings.csv')
movies_df = pd.read_csv('MovieLens_100k/movies.csv')

# Data Preprocessing
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

ratings_df['user'] = user_encoder.fit_transform(ratings_df['userId'])
ratings_df['item'] = item_encoder.fit_transform(ratings_df['movieId'])

num_users = ratings_df['user'].nunique()
num_items = ratings_df['item'].nunique()

def evaluate_model(model, train_data, test_data, adj_matrix=None):
    # Prepare the training data
    user_indices = train_data['user'].values
    item_indices = train_data['item'].values
    labels = train_data['rating'].values

    dataset = tf.data.Dataset.from_tensor_slices((user_indices, item_indices, labels))
    dataset = dataset.shuffle(buffer_size=len(train_data)).batch(256)

    # Training loop
    optimizer = tf.keras.optimizers.Adam()
    loss_fn = tf.keras.losses.MeanSquaredError()  

    @tf.function
    def train_step(user_indices, item_indices, labels):
        with tf.GradientTape() as tape:
            scores = model((user_indices, item_indices))
            loss = loss_fn(labels, scores)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return loss

    num_epochs = 10
    start_time = time.time()

    for epoch in range(num_epochs):
        for batch in dataset:
            user_indices_batch, item_indices_batch, labels_batch = batch
            loss = train_step(user_indices_batch, item_indices_batch, labels_batch)
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')

    training_time = time.time() - start_time

    # Evaluating the model
    predictions = []

    test_user_indices = test_data['user'].values
    test_item_indices = test_data['item'].values
    test_labels = test_data['rating'].values

    for (user_index, item_index, label) in zip(test_user_indices, test_item_indices, test_labels):
        user_index_tensor = tf.constant([user_index])
        item_index_tensor = tf.constant([item_index])
        score = model((user_index_tensor, item_index_tensor)).numpy()[0]
        predictions.append((user_index, item_index, label, score, 0))

    return predictions, training_time

def compute_metrics(predictions):
    def get_top_n(predictions, n=10):
        top_n = defaultdict(list)
        for uid, iid, true_r, est, _ in predictions:
            top_n[uid].append((iid, est))
        for uid, user_ratings in top_n.items():
            user_ratings.sort(key=lambda x: x[1], reverse=True)
            top_n[uid] = user_ratings[:n]
        return top_n

    def precision_recall_at_k(predictions, k=10, threshold=3.5): 
        user_est_true = defaultdict(list)
        for uid, _, true_r, est, _ in predictions:
            user_est_true[uid].append((est, true_r))
        precisions = dict()
        recalls = dict()
        for uid, user_ratings in user_est_true.items():
            user_ratings.sort(key=lambda x: x[0], reverse=True)
            n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
            n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
            n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])
            precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
            recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
        return precisions, recalls

    def compute_mse(predictions):
        """Compute Mean Squared Error (MSE)."""
        mse = np.mean([(true_r - est) ** 2 for (_, _, true_r, est, _) in predictions])
        return mse

    def compute_rmse(predictions):
        """Compute Root Mean Squared Error (RMSE)."""
        mse = compute_mse(predictions)
        rmse = np.sqrt(mse)
        return rmse

    def compute_mae(predictions):
        mae = np.mean([abs(true_r - est) for (_, _, true_r, est, _) in predictions])
        return mae

    mse = compute_mse(predictions)
    rmse = compute_rmse(predictions)
    mae = compute_mae(predictions)
    precisions, recalls = precision_recall_at_k(predictions, k=10)

    precision_at_10 = np.mean(list(precisions.values()))
    recall_at_10 = np.mean(list(recalls.values()))

    return mse, rmse, mae, precision_at_10, recall_at_10

# Function to run evaluation for each model and scenario
def run_evaluation(model, model_name, train_data, test_data, adj_matrix=None):
    predictions, training_time = evaluate_model(model, train_data, test_data, adj_matrix)
    mse, rmse, mae, precision_at_10, recall_at_10 = compute_metrics(predictions)
    results = pd.DataFrame({
        "Scenario": [scenario],
        "Algorithm": [model_name],
        "MSE": [mse],
        "RMSE": [rmse],
        "MAE": [mae],
        "Precision@10": [precision_at_10],
        "Recall@10": [recall_at_10],
        "Running Time (s)": [training_time]
    })
    return results

# Evaluate LightGCN
class LightGCN(tf.keras.Model):
    def __init__(self, num_users, num_items, embedding_dim, **kwargs):
        super(LightGCN, self).__init__(**kwargs)
        self.user_embedding = tf.keras.layers.Embedding(num_users, embedding_dim)
        self.item_embedding = tf.keras.layers.Embedding(num_items, embedding_dim)
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim

    def call(self, inputs):
        user_indices, item_indices = inputs
        user_embeddings = self.user_embedding(user_indices)
        item_embeddings = self.item_embedding(item_indices)
        scores = tf.reduce_sum(user_embeddings * item_embeddings, axis=1)
        return scores

    def get_config(self):
        config = super(LightGCN, self).get_config()
        config.update({
            'num_users': self.num_users,
            'num_items': self.num_items,
            'embedding_dim': self.embedding_dim,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

embedding_dim = 64
lightgcn_model = LightGCN(num_users, num_items, embedding_dim)

# Step 1: Split the data into training and test sets (80% training, 20% testing)
train_df, test_df = train_test_split(ratings_df, test_size=0.20, random_state=42)

# Callbacks for early stopping and model checkpointing
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')

# Compile the model
lightgcn_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with early stopping and model checkpointing
history = lightgcn_model.fit(
    x=(train_df['user'].values, train_df['item'].values),
    y=train_df['rating'].values,
    epochs=10,  
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stopping, checkpoint]
)

# Load the best model
best_model = tf.keras.models.load_model('best_model.keras', custom_objects={'LightGCN': LightGCN})

def get_sparse_data(ratings, frac=0.1):
    # Placeholder function to get sparse data. Replace with actual logic.
    return ratings.sample(frac=frac, random_state=42)

def get_new_user_data(ratings, frac=0.1):
    # Placeholder function to get new user data. Replace with actual logic.
    new_user_indices = ratings['user'].drop_duplicates().sample(frac=frac, random_state=42).index
    return ratings[ratings['user'].isin(new_user_indices)]

# Evaluate normal scenario
scenario = "Normal"
results_lightGCN_normal = run_evaluation(best_model, "LightGCN", train_df, test_df)

# Evaluate sparse scenario
scenario = "Sparse"
sparse_train_df = get_sparse_data(train_df, frac=0.1)
results_lightGCN_sparse = run_evaluation(best_model, "LightGCN", sparse_train_df, test_df)

# Evaluate new user scenario
scenario = "New User"
new_user_train_df = get_new_user_data(train_df, frac=0.1)
results_lightGCN_new_user = run_evaluation(best_model, "LightGCN", new_user_train_df, test_df)

# Combine LightGCN results into a single DataFrame
results_lightGCN_combined = pd.concat([results_lightGCN_normal, results_lightGCN_sparse, results_lightGCN_new_user], ignore_index=True)

# Display the combined results
print(results_lightGCN_combined)

Epoch 1/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 13.2447 - val_loss: 9.8584
Epoch 2/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 6.6229 - val_loss: 2.7912
Epoch 3/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 1.9666 - val_loss: 1.9016
Epoch 4/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 1.1804 - val_loss: 1.6163
Epoch 5/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.8587 - val_loss: 1.4916
Epoch 6/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.7028 - val_loss: 1.4394
Epoch 7/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.6087 - val_loss: 1.4152
Epoch 8/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.5412 - val_loss: 1.4058
Epoch 9/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4819 - val_loss: 1.4033
Epoch 10/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4406 - val_loss: 1.4041


2024-08-09 15:11:52.903066: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 0, Loss: 1.0922755002975464


2024-08-09 15:11:54.316141: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1, Loss: 0.524085521697998


...

Epoch 6, Loss: 0.06872354447841644
Epoch 7, Loss: 0.08671410381793976


2024-08-09 15:13:18.159858: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-09 15:13:18.306919: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 8, Loss: 0.05715125799179077
Epoch 9, Loss: 0.06465554982423782


2024-08-09 15:23:31.950517: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-09 15:23:31.957898: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-09 15:23:31.964924: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-09 15:23:31.971387: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-09 15:23:31.979132: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-09 15:23:31.986603: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-09 15:23:31.994938: W tensorflow/core/framework/local_rendezvous.cc:404] L

Epoch 0, Loss: 0.2316470891237259
Epoch 1, Loss: 0.20376814901828766
Epoch 2, Loss: 0.1803845763206482
Epoch 3, Loss: 0.16075095534324646
Epoch 4, Loss: 0.1441202163696289
Epoch 5, Loss: 0.12986089289188385
Epoch 6, Loss: 0.11745857447385788
Epoch 7, Loss: 0.10649746656417847
Epoch 8, Loss: 0.09666289389133453
Epoch 9, Loss: 0.08773539960384369
   Scenario Algorithm       MSE      RMSE       MAE  Precision@10  Recall@10  \
0    Normal  LightGCN  1.300472  1.140382  0.814420      0.749278   0.503184   
1    Sparse  LightGCN  1.353399  1.163357  0.837602      0.750687   0.504528   
2  New User  LightGCN  1.352309  1.162888  0.837188      0.751206   0.504964   

   Running Time (s)  
0         16.588426  
1          2.137099  
2          0.539947  


In [15]:
# Combine LightGCN results into a single DataFrame
results_lightGCN_combined = pd.concat([results_lightGCN_normal, results_lightGCN_sparse, results_lightGCN_new_user], ignore_index=True)
results_lightGCN_combined

,Scenario,Algorithm,MSE,RMSE,MAE,Precision@10,Recall@10,Running Time (s)
0,Normal,LightGCN,1.300472,1.140382,0.814420,0.749278,0.503184,16.588426
1,Sparse,LightGCN,1.353399,1.163357,0.837602,0.750687,0.504528,2.137099
2,New User,LightGCN,1.352309,1.162888,0.837188,0.751206,0.504964,0.539947


# Graph Attention Network (GAT)

In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf
import time
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load the dataset
ratings_df = pd.read_csv('MovieLens_100k/ratings.csv')
movies_df = pd.read_csv('MovieLens_100k/movies.csv')

# Data Preprocessing
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

ratings_df['user'] = user_encoder.fit_transform(ratings_df['userId'])
ratings_df['item'] = item_encoder.fit_transform(ratings_df['movieId'])

num_users = ratings_df['user'].nunique()
num_items = ratings_df['item'].nunique()

def evaluate_model(model, train_data, test_data):
    # Prepare the training data
    user_indices = train_data['user'].values
    item_indices = train_data['item'].values
    labels = train_data['rating'].values

    dataset = tf.data.Dataset.from_tensor_slices((user_indices, item_indices, labels))
    dataset = dataset.shuffle(buffer_size=len(train_data)).batch(256)

    # Training loop
    optimizer = tf.keras.optimizers.Adam()
    loss_fn = tf.keras.losses.MeanSquaredError()  

    @tf.function
    def train_step(user_indices, item_indices, labels):
        with tf.GradientTape() as tape:
            scores = model((user_indices, item_indices))
            loss = loss_fn(labels, scores)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return loss

    num_epochs = 10
    start_time = time.time()

    for epoch in range(num_epochs):
        for batch in dataset:
            user_indices_batch, item_indices_batch, labels_batch = batch
            loss = train_step(user_indices_batch, item_indices_batch, labels_batch)
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')

    training_time = time.time() - start_time

    # Evaluating the model
    predictions = []

    test_user_indices = test_data['user'].values
    test_item_indices = test_data['item'].values
    test_labels = test_data['rating'].values

    for (user_index, item_index, label) in zip(test_user_indices, test_item_indices, test_labels):
        user_index_tensor = tf.constant([user_index])
        item_index_tensor = tf.constant([item_index])
        score = model((user_index_tensor, item_index_tensor)).numpy()[0]
        predictions.append((user_index, item_index, label, score, 0))

    return predictions, training_time

def compute_metrics(predictions):
    def get_top_n(predictions, n=10):
        top_n = defaultdict(list)
        for uid, iid, true_r, est, _ in predictions:
            top_n[uid].append((iid, est))
        for uid, user_ratings in top_n.items():
            user_ratings.sort(key=lambda x: x[1], reverse=True)
            top_n[uid] = user_ratings[:n]
        return top_n

    def precision_recall_at_k(predictions, k=10, threshold=3.5): 
        user_est_true = defaultdict(list)
        for uid, _, true_r, est, _ in predictions:
            user_est_true[uid].append((est, true_r))
        precisions = dict()
        recalls = dict()
        for uid, user_ratings in user_est_true.items():
            user_ratings.sort(key=lambda x: x[0], reverse=True)
            n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
            n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
            n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])
            precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
            recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
        return precisions, recalls

    def compute_mse(predictions):
        """Compute Mean Squared Error (MSE)."""
        mse = np.mean([(true_r - est) ** 2 for (_, _, true_r, est, _) in predictions])
        return mse

    def compute_rmse(predictions):
        """Compute Root Mean Squared Error (RMSE)."""
        mse = compute_mse(predictions)
        rmse = np.sqrt(mse)
        return rmse

    def compute_mae(predictions):
        mae = np.mean([abs(true_r - est) for (_, _, true_r, est, _) in predictions])
        return mae

    mse = compute_mse(predictions)
    rmse = compute_rmse(predictions)
    mae = compute_mae(predictions)
    precisions, recalls = precision_recall_at_k(predictions, k=10)

    precision_at_10 = np.mean(list(precisions.values()))
    recall_at_10 = np.mean(list(recalls.values()))

    return mse, rmse, mae, precision_at_10, recall_at_10

# Function to run evaluation for each model and scenario
def run_evaluation(model, model_name, train_data, test_data):
    predictions, training_time = evaluate_model(model, train_data, test_data)
    mse, rmse, mae, precision_at_10, recall_at_10 = compute_metrics(predictions)
    results = pd.DataFrame({
        "Scenario": [scenario],
        "Algorithm": [model_name],
        "MSE": [mse],
        "RMSE": [rmse],
        "MAE": [mae],
        "Precision@10": [precision_at_10],
        "Recall@10": [recall_at_10],
        "Running Time (s)": [training_time]
    })
    return results

# Evaluate GAT
class GraphAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, output_dim, attn_heads=1, dropout_rate=0.0, **kwargs):
        super(GraphAttentionLayer, self).__init__(**kwargs)
        self.output_dim = output_dim
        self.attn_heads = attn_heads
        self.dropout_rate = dropout_rate
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.attn_kernels = []
        self.attn_self_kernels = []

        for _ in range(attn_heads):
            self.attn_kernels.append(tf.keras.layers.Dense(output_dim, use_bias=False))
            self.attn_self_kernels.append(tf.keras.layers.Dense(1, use_bias=False))

    def call(self, inputs):
        features, adj_matrix = inputs
        attn_outs = []

        for kernel, self_kernel in zip(self.attn_kernels, self.attn_self_kernels):
            attn_out = kernel(features)
            attn_self = self_kernel(features)
            attn_all = tf.add(attn_self, tf.transpose(attn_self))
            attn_all = tf.nn.leaky_relu(attn_all)
            attn_all = tf.nn.softmax(attn_all, axis=-1)
            attn_all = self.dropout(attn_all)
            node_features = tf.matmul(attn_all, attn_out)
            attn_outs.append(node_features)

        return tf.concat(attn_outs, axis=-1)

    def get_config(self):
        config = super(GraphAttentionLayer, self).get_config()
        config.update({
            'output_dim': self.output_dim,
            'attn_heads': self.attn_heads,
            'dropout_rate': self.dropout_rate,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

class GATModel(tf.keras.Model):
    def __init__(self, num_users, num_items, embedding_dim, attn_heads=1, dropout_rate=0.0, **kwargs):
        super(GATModel, self).__init__(**kwargs)
        self.user_embedding = tf.keras.layers.Embedding(num_users, embedding_dim)
        self.item_embedding = tf.keras.layers.Embedding(num_items, embedding_dim)
        self.gat_layer = GraphAttentionLayer(embedding_dim, attn_heads, dropout_rate)
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.attn_heads = attn_heads
        self.dropout_rate = dropout_rate
        self.adj_matrix = None

    def set_adj_matrix(self, adj_matrix):
        self.adj_matrix = adj_matrix

    def call(self, inputs):
        user_indices, item_indices = inputs
        user_embeddings = self.user_embedding(user_indices)
        item_embeddings = self.item_embedding(item_indices)
        all_embeddings = tf.concat([user_embeddings, item_embeddings], axis=0)
        all_embeddings = self.gat_layer([all_embeddings, self.adj_matrix])
        user_embeddings = all_embeddings[:tf.shape(user_indices)[0]]
        item_embeddings = all_embeddings[tf.shape(user_indices)[0]:]
        scores = tf.reduce_sum(user_embeddings * item_embeddings, axis=1)
        return scores

    def get_config(self):
        config = super(GATModel, self).get_config()
        config.update({
            'num_users': self.num_users,
            'num_items': self.num_items,
            'embedding_dim': self.embedding_dim,
            'attn_heads': self.attn_heads,
            'dropout_rate': self.dropout_rate,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

adj_matrix = np.zeros((num_users + num_items, num_users + num_items))
for _, row in ratings_df.iterrows():
    user_id = int(row['user'])
    item_id = int(row['item']) + num_users
    adj_matrix[user_id, item_id] = 1
    adj_matrix[item_id, user_id] = 1

adj_matrix = tf.convert_to_tensor(adj_matrix, dtype=tf.float32)

embedding_dim = 64
attn_heads = 4
dropout_rate = 0.5
gat_model = GATModel(num_users, num_items, embedding_dim, attn_heads, dropout_rate)
gat_model.set_adj_matrix(adj_matrix)

# Callbacks for early stopping and model checkpointing
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint('best_gat_model.keras', monitor='val_loss', save_best_only=True, mode='min')

# Compile the model
gat_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with early stopping and model checkpointing
history = gat_model.fit(
    x=[train_df['user'].values, train_df['item'].values],
    y=train_df['rating'].values,
    epochs=10,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stopping, checkpoint]
)

# Load the best model
best_gat_model = tf.keras.models.load_model('best_gat_model.keras', custom_objects={'GATModel': GATModel, 'GraphAttentionLayer': GraphAttentionLayer})
best_gat_model.set_adj_matrix(adj_matrix)


# Evaluate normal scenario
scenario = "Normal"
results_GAT_normal = run_evaluation(best_gat_model, "GAT", train_df, test_df)

# Evaluate sparse scenario
scenario = "Sparse"
sparse_train_df = get_sparse_data(train_df, frac=0.1)
results_GAT_sparse = run_evaluation(best_gat_model, "GAT", sparse_train_df, test_df)

# Evaluate new user scenario
scenario = "New User"
new_user_train_df = get_new_user_data(train_df, frac=0.1)
results_GAT_new_user = run_evaluation(best_gat_model, "GAT", new_user_train_df, test_df)

# Combine GAT results into a single DataFrame
results_GAT_combined = pd.concat([results_GAT_normal, results_GAT_sparse, results_GAT_new_user], ignore_index=True)

# Display the combined results
print(results_GAT_combined)

Epoch 1/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - loss: 4.4186 - val_loss: 1.1508
Epoch 2/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 1.1240 - val_loss: 1.1469
Epoch 3/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 1.1112 - val_loss: 1.0692
Epoch 4/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 1.0319 - val_loss: 1.0472
Epoch 5/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.9298 - val_loss: 0.9864
Epoch 6/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.9177 - val_loss: 1.0135
Epoch 7/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.8428 - val_loss: 0.9277
Epoch 8/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.8204 - val_loss: 0.9054
Epoch 9/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.7728 - val_loss: 0.8885
Epoch 10/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.7501 - val_loss: 0.8999


/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/serialization_lib.py:730: UserWarning: Model 'gat_model_3' had a build config, but the model cannot be built automatically in `build_from_config(config)`. You should implement `def build_from_config(self, config)`, and you might also want to implement the method  that generates the config at saving time, `def get_build_config(self)`. The method `build_from_config()` is meant to create the state of the model (i.e. its variables) upon deserialization.
  instance.build_from_config(build_config)
/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 6 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
2024-08-09 13:33:08.085953: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 0, Loss: 0.8535671234130859


2024-08-09 13:33:15.026881: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1, Loss: 0.791020929813385


...

Epoch 8, Loss: 0.5348884463310242


2024-08-09 13:38:43.371272: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 9, Loss: 0.5008713603019714


2024-08-09 13:42:57.508671: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-09 13:42:57.524851: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-09 13:42:57.536148: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-09 13:42:57.547046: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-09 13:42:57.558032: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-09 13:42:57.569295: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-09 13:42:57.580212: W tensorflow/core/framework/local_rendezvous.cc:404] L

Epoch 0, Loss: 0.5133550763130188
Epoch 1, Loss: 0.4729280471801758
Epoch 2, Loss: 0.5329741835594177
Epoch 3, Loss: 0.32601064443588257
Epoch 4, Loss: 0.2739917039871216
Epoch 5, Loss: 0.3939181864261627
Epoch 6, Loss: 0.3761257827281952
Epoch 7, Loss: 0.27111342549324036
Epoch 8, Loss: 0.24785055220127106
Epoch 9, Loss: 0.29781031608581543
   Scenario Algorithm       MSE      RMSE       MAE  Precision@10  Recall@10  \
0    Normal       GAT  5.279527  2.297722  1.813445      0.640868   0.339237   
1    Sparse       GAT  9.903407  3.146968  2.508598      0.665226   0.375002   
2  New User       GAT  8.366129  2.892426  2.294709      0.656693   0.358051   

   Running Time (s)  
0         76.762282  
1          9.373171  
2          2.174327  


In [3]:
results_GAT_combined

,Scenario,Algorithm,MSE,RMSE,MAE,Precision@10,Recall@10,Running Time (s)
0,Normal,GAT,5.279527,2.297722,1.813445,0.640868,0.339237,76.762282
1,Sparse,GAT,9.903407,3.146968,2.508598,0.665226,0.375002,9.373171
2,New User,GAT,8.366129,2.892426,2.294709,0.656693,0.358051,2.174327


# GraphSAGE (SAmple and aggreGatE)

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from scipy.sparse import lil_matrix, csr_matrix

# Load the dataset
ratings_df = pd.read_csv('MovieLens_100k/ratings.csv')
movies_df = pd.read_csv('MovieLens_100k/movies.csv')

# Data Preprocessing
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

ratings_df['user'] = user_encoder.fit_transform(ratings_df['userId'])
ratings_df['item'] = item_encoder.fit_transform(ratings_df['movieId'])

num_users = ratings_df['user'].nunique()
num_items = ratings_df['item'].nunique()

# Adjacency Matrix Construction
adj_matrix = lil_matrix((num_users + num_items, num_users + num_items))
for _, row in ratings_df.iterrows():
    user_id = int(row['user'])
    item_id = int(row['item']) + num_users
    adj_matrix[user_id, item_id] = 1
    adj_matrix[item_id, user_id] = 1

adj_matrix = csr_matrix(adj_matrix)

# Convert to TensorFlow SparseTensor
adj_matrix_indices = np.vstack((adj_matrix.nonzero()[0], adj_matrix.nonzero()[1])).T
adj_matrix_values = adj_matrix.data
adj_matrix_shape = adj_matrix.shape

adj_matrix = tf.sparse.SparseTensor(
    indices=adj_matrix_indices,
    values=adj_matrix_values,
    dense_shape=adj_matrix_shape
)

# Ensure the sparse tensor is properly ordered
adj_matrix = tf.sparse.reorder(adj_matrix)

class GraphSAGELayer(tf.keras.layers.Layer):
    def __init__(self, output_dim, aggregator_type='mean', dropout_rate=0.0, **kwargs):
        super(GraphSAGELayer, self).__init__(**kwargs)
        self.output_dim = output_dim
        self.aggregator_type = aggregator_type
        self.dropout_rate = dropout_rate
        self.dense_self = tf.keras.layers.Dense(output_dim, use_bias=False)
        self.dense_neighbor = tf.keras.layers.Dense(output_dim, use_bias=False)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.act = tf.nn.relu

    def call(self, inputs):
        features, adj_matrix = inputs
        neighbor_features = tf.sparse.sparse_dense_matmul(adj_matrix, features)
        node_features = self.dense_self(features) + self.dense_neighbor(neighbor_features)
        node_features = self.act(node_features)
        node_features = self.dropout(node_features)
        return node_features

class GraphSAGEModel(tf.keras.Model):
    def __init__(self, num_users, num_items, embedding_dim, aggregator_type='mean', dropout_rate=0.0, **kwargs):
        super(GraphSAGEModel, self).__init__(**kwargs)
        self.user_embedding = tf.keras.layers.Embedding(num_users, embedding_dim)
        self.item_embedding = tf.keras.layers.Embedding(num_items, embedding_dim)
        self.graphsage_layer = GraphSAGELayer(embedding_dim, aggregator_type, dropout_rate)
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.aggregator_type = aggregator_type
        self.dropout_rate = dropout_rate
        self.adj_matrix = None

    def set_adj_matrix(self, adj_matrix):
        self.adj_matrix = adj_matrix

    def call(self, inputs):
        user_indices, item_indices = inputs
        user_embeddings = self.user_embedding(user_indices)
        item_embeddings = self.item_embedding(item_indices)
        all_user_embeddings = self.user_embedding(tf.range(self.num_users))
        all_item_embeddings = self.item_embedding(tf.range(self.num_items))
        all_embeddings = tf.concat([all_user_embeddings, all_item_embeddings], axis=0)
        all_embeddings = self.graphsage_layer([all_embeddings, self.adj_matrix])
        user_embeddings = tf.gather(all_embeddings, user_indices)
        item_embeddings = tf.gather(all_embeddings, item_indices + self.num_users)
        scores = tf.reduce_sum(user_embeddings * item_embeddings, axis=1)
        return scores

    def get_config(self):
        config = super(GraphSAGEModel, self).get_config()
        config.update({
            'num_users': self.num_users,
            'num_items': self.num_items,
            'embedding_dim': self.embedding_dim,
            'aggregator_type': self.aggregator_type,
            'dropout_rate': self.dropout_rate,
        })
        return config

# Set the adjacency matrix
embedding_dim = 64
aggregator_type = 'mean'
dropout_rate = 0.5
graphsage_model = GraphSAGEModel(num_users, num_items, embedding_dim, aggregator_type, dropout_rate)
graphsage_model.set_adj_matrix(adj_matrix)

# Callbacks for early stopping and model checkpointing
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint('best_graphsage_model.keras', monitor='val_loss', save_best_only=True, mode='min')

# Compile the model
graphsage_model.compile(optimizer='adam', loss='mean_squared_error')

# Prepare training and testing data (assuming train_df and test_df are predefined)
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=42)

# Train the model with early stopping and model checkpointing
history = graphsage_model.fit(
    x=[train_df['user'].values, train_df['item'].values],
    y=train_df['rating'].values,
    epochs=10,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stopping, checkpoint]
)

# Load the best model
best_graphsage_model = tf.keras.models.load_model('best_graphsage_model.keras', custom_objects={'GraphSAGEModel': GraphSAGEModel, 'GraphSAGELayer': GraphSAGELayer})
best_graphsage_model.set_adj_matrix(adj_matrix)

# Evaluate normal scenario
scenario = "Normal"
results_SAGE_normal = run_evaluation(best_graphsage_model, "GraphSAGE", train_df, test_df)

Epoch 1/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - loss: 3.2163 - val_loss: 1.5399
Epoch 2/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - loss: 1.1261 - val_loss: 1.1178
Epoch 3/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - loss: 0.8146 - val_loss: 1.0694
Epoch 4/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - loss: 0.7007 - val_loss: 1.0774
Epoch 5/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - loss: 0.6651 - val_loss: 1.1709
Epoch 6/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 14s 27ms/step - loss: 0.6228 - val_loss: 1.1076


/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/serialization_lib.py:730: UserWarning: Model 'graph_sage_model_2' had a build config, but the model cannot be built automatically in `build_from_config(config)`. You should implement `def build_from_config(self, config)`, and you might also want to implement the method  that generates the config at saving time, `def get_build_config(self)`. The method `build_from_config()` is meant to create the state of the model (i.e. its variables) upon deserialization.
  instance.build_from_config(build_config)
/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
2024-08-08 20:33:48.929446: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 0, Loss: 0.8244467973709106


2024-08-08 20:33:55.527243: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1, Loss: 0.8825513124465942


...

Epoch 7, Loss: 1.2692623138427734


2024-08-08 20:34:44.164126: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 8, Loss: 0.6173475980758667


2024-08-08 20:34:51.570223: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 9, Loss: 0.24220798909664154
  Scenario  Algorithm       MSE      RMSE       MAE  Precision@10  Recall@10  \
0   Normal  GraphSAGE  0.995274  0.997634  0.765923      0.737043   0.525224   

   Running Time (s)  
0         71.149196  


In [8]:
# Evaluate sparse scenario
scenario = "Sparse"
sparse_train_df = get_sparse_data(train_df, frac=0.1)
results_SAGE_sparse = run_evaluation(best_graphsage_model, "GraphSAGE", sparse_train_df, test_df)

# Evaluate new user scenario
scenario = "New User"
new_user_train_df = get_new_user_data(train_df, frac=0.1)
results_SAGE_new_user = run_evaluation(best_graphsage_model, "GraphSAGE", new_user_train_df, test_df)

2024-08-08 20:41:40.752479: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 0, Loss: 0.5703974366188049


2024-08-08 20:41:41.434409: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1, Loss: 0.509819746017456


2024-08-08 20:41:42.130202: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2024-08-08 20:48:22.836179: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-08 20:48:22.862791: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-08 20:48:22.887528: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-08 20:48:22.914367: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-08 20:48:22.937209: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-08 20:48:22.959044: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-08 20:48:22.981578: W tensorflow/core/framework/local_rendezvous.cc:404] L

Epoch 0, Loss: 0.44780388474464417
Epoch 1, Loss: 3.4853625297546387
Epoch 2, Loss: 0.2548128366470337
Epoch 3, Loss: 0.9237936735153198
Epoch 4, Loss: 1.4512138366699219
Epoch 5, Loss: 1.2539323568344116
Epoch 6, Loss: 0.6974046230316162
Epoch 7, Loss: 0.308746874332428
Epoch 8, Loss: 0.47734466195106506


2024-08-08 20:48:23.061360: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 9, Loss: 0.7331115007400513


,Scenario,Algorithm,MSE,RMSE,MAE,Precision@10,Recall@10,Running Time (s)
0,NormalSparseNew User,GraphSAGEGraphSAGEGraphSAGE,4.539952,3.623258,2.8031,2.19935,1.495469,80.491525


In [9]:
results_SAGE_combined = pd.concat([results_SAGE_normal, results_SAGE_sparse, results_SAGE_new_user], ignore_index=True)
results_SAGE_combined

,Scenario,Algorithm,MSE,RMSE,MAE,Precision@10,Recall@10,Running Time (s)
0,Normal,GraphSAGE,0.995274,0.997634,0.765923,0.737043,0.525224,71.149196
1,Sparse,GraphSAGE,1.191938,1.091759,0.853637,0.729891,0.473050,8.101512
2,New User,GraphSAGE,2.352740,1.533864,1.183540,0.732416,0.497194,1.240817


## Final Result 

In [15]:
combined_results = pd.concat([results_LightGCN_combined, results_GAT_combined, results_GraphSAGE_combined], ignore_index=True)
combined_results

,Scenario,Algorithm,MSE,RMSE,MAE,Precision@10,Recall@10,Running Time (s)
0,Normal,LightGCN,1.300472,1.140382,0.814420,0.749278,0.503184,16.588426
1,Sparse,LightGCN,1.353399,1.163357,0.837602,0.750687,0.504528,2.137099
2,New User,LightGCN,1.352309,1.162888,0.837188,0.751206,0.504964,0.539947
3,Normal,GAT,5.279527,2.297722,1.813445,0.640868,0.339237,76.762282
4,Sparse,GAT,9.903407,3.146968,2.508598,0.665226,0.375002,9.373171
5,New User,GAT,8.366129,2.892426,2.294709,0.656693,0.358051,2.174327
6,Normal,GraphSAGE,0.995274,0.997634,0.765923,0.737043,0.525224,71.149196
7,Sparse,GraphSAGE,1.191938,1.091759,0.853637,0.729891,0.473050,8.101512
8,New User,GraphSAGE,2.352740,1.533864,1.183540,0.732416,0.497194,1.240817
